In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Оценка качества кофе

* Aroma: Относится к аромату или благоуханию кофе.
* Flavor: Аромат кофе оценивается на основе вкуса, включая любую сладость, горечь, кислотность и другие вкусовые нотки.
* Aftertaste: Относится к затяжному вкусу, который остается во рту после проглатывания кофе.
* Acidity: Кислотность в кофе относится к яркости или живости вкуса.
* Body: Консистенция кофе относится к густоте или вязкости кофе во рту.
* Balance: Баланс относится к тому, насколько хорошо различные вкусовые компоненты кофе сочетаются друг с другом.
* Uniformity: Однородность относится к консистенции кофе от чашки к чашке.
* Clean Cup: Под чистой чашкой подразумевается кофе, в котором отсутствуют какие-либо посторонние привкусы или дефекты, такие как кислинка, затхлость или несвежесть.
* Sweetness: Ее можно охарактеризовать как карамельную, фруктовую или цветочную, и она является желательным качеством кофе.


# Дефекты

Дефекты - это нежелательные качества, которые могут проявляться в кофейных зернах во время обработки или хранения. Дефекты можно разделить на две категории: дефекты первой и второй категорий.

1. Дефекты первой категории - это первичные дефекты, которые можно обнаружить при визуальном осмотре кофейных зерен. К таким дефектам относятся черные бобы, кислые бобы, поврежденные насекомыми бобы, поврежденные грибками бобы и т.д.

2. Дефекты второй категории - это вторичные дефекты, которые являются более тонкими и могут быть обнаружены только при дегустации. К таким дефектам относятся чрезмерное брожение, несвежесть, прогорклость, химический привкус и т.д.

In [ ]:
coffe_df = pd.read_csv('/kaggle/input/coffee-quality-data-cqi/df_arabica_clean.csv')

In [ ]:
coffe_df.head()

In [ ]:
coffe_df.info()

Датасет небольшой, поэтому удалять данные нерационально, так как это может значительно может повлиять на результаты анализа

In [ ]:
coffe_df['Expiration']

In [ ]:
# Удаляем ненужные категории (Unnamed: 0, ID)
# В категории ICo Number много пропущенных значений, поэтому тоже удаляем
coffe_df.drop(labels=['Unnamed: 0', 'ID', 'ICO Number'], axis=1, inplace=True)

Заменим пропущенные категориальные признаки наиболее часто встречающимися значениями

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent")
val_list = coffe_df.select_dtypes(include=["object"]).columns.tolist()
coffe_df[val_list] = imputer.fit_transform(coffe_df[val_list])

Заменим тип данных в некоторых категориях

In [ ]:
coffe_df['Bag Weight'] = coffe_df['Bag Weight'].str.split().str.get(0).astype(int)
coffe_df['Harvest Year'] = coffe_df['Harvest Year'].str.split('/').str.get(0).astype(int)
coffe_df['Grading Date'] = coffe_df['Grading Date'].str.split(',').str.get(1).astype(int) # берем год
coffe_df['Expiration'] = coffe_df['Expiration'].str.split(',').str.get(1).astype(int) # берем год

Также удалим некоторые категории данных:

* Certification Contact не связан с общим количеством очков Кубка
* Certificaion Address будет удален, поскольку у Органа по сертификации есть название страны, в которой находится адрес сертификации
* Lot Number не оказывает никакого влияния на целевую переменную.
* Altitude будет снижена, потому что у нас будет страна происхождения
* Farm Name и Mill имеют почти одинаковое значение, поэтому Mill будет удалена.
* Status имеет только одно значение 1, поэтому он будет удален.

Aroma, Flavor, Aftertaste, Acidity, Body, Balance, Uniformity, Clean Cup, Sweetness,  Overall: сумма этих характеристик составляет общее количество баллов за чашку (Total Cup Points). Итак, теперь мы отбросим эти упомянутые столбцы

In [ ]:
drop_1 = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean Cup', 'Sweetness', 'Overall']
coffe_df.drop(labels=drop_1, axis=1, inplace=True)

Значение параметра **Total Cup Points** - основной оцениваемый параметр. Поэтому категории, которые плохо коррелируемы с этим параметром нам не подходят.

In [ ]:
# Прокоррелируем все значения и оценим корреляцию параметров с Total Cup Points
corr_matrix = coffe_df.corr(numeric_only=True)
corr_matrix["Total Cup Points"].sort_values(ascending=False)

Наиболее коррелируемыми критериями являются **Quakers** и **Category Two Defects**. Оставляем данные параметры, а остальные удаляем

In [ ]:
drop_2 = ['Defects', 'Bag Weight', 'Moisture Percentage', 'Category One Defects', 'Harvest Year', 'Grading Date', 'Expiration', 'Number of Bags']
coffe_df.drop(labels=drop_2, axis=1, inplace=True)

In [ ]:
# Проверка, чтобы не было пропущенных значений
coffe_df.isna().sum()

# Визуализация

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
corr_matrix_2 = coffe_df.corr(numeric_only=True)
corr_matrix_2["Total Cup Points"].sort_values(ascending=False)

In [ ]:
sns.heatmap(corr_matrix_2, annot=True, cbar=True, fmt=".2f", cmap="crest")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
f = sns.pairplot(data=coffe_df, hue="Total Cup Points")
f.fig.set_size_inches(10, 5)
plt.show()

In [ ]:
# Разделим категориальные и числовые категории
cat_list = coffe_df.select_dtypes(include=["object"]).columns.tolist()
num_list = coffe_df.select_dtypes(exclude=["object"]).columns.tolist()

In [ ]:
for col in cat_list:
    #print(col)
    plt.figure(figsize=(10,6))
    top_10_val = coffe_df[col].value_counts()[:10]
    top_10_val.plot(kind='bar')
    plt.title("Top 10 " + col)
    plt.grid(visible=False)
    plt.show()

# Разделяем данные на тестовую и тренировочную части

In [ ]:
from sklearn.model_selection import train_test_split

X = coffe_df.drop(labels='Total Cup Points', axis=1)
y = coffe_df['Total Cup Points']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [ ]:
num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", MinMaxScaler())])
cat_pipeline = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("hot_encoder", OneHotEncoder(handle_unknown="ignore"))])

In [ ]:
cat_list = X_train.select_dtypes(include=["object"]).columns.tolist()
num_list = X_train.select_dtypes(exclude=["object"]).columns.tolist()

In [ ]:
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([("num", num_pipeline, num_list), ("category", cat_pipeline, cat_list),])

In [ ]:
from sklearn import set_config

set_config(display="diagram")
full_pipeline

In [ ]:
X_train_norm = full_pipeline.fit_transform(X_train)
X_test_norm = full_pipeline.transform(X_test)

В качестве модели используется **RandomForestRegressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=10, max_features=250)
forest.fit(X_train_norm, y_train)

y_pred = forest.predict(full_pipeline.transform(X_test))
y_t = forest.predict(full_pipeline.transform(X_train))

In [ ]:
# Оценим полученную модель
from sklearn.metrics import mean_squared_error, r2_score

r2_score(y_test, y_pred), r2_score(y_t, y_train)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mse

In [ ]:
# Визуализация предикта
plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test, y=y_pred, hue=abs(y_test - y_pred), s=80) 
plt.xlabel("Actual Cup Points", fontsize=20)
plt.ylabel("Predicted Cup Points", fontsize=20)
plt.show()